In [325]:
%reset -f
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
import math
import itertools

almNum = 3
ocorr = 10
t_sample = 60
base_hour = dt.datetime(2018, 1,9,9,0,0)
occor_num = 0

#Settings alarms
a = np.zeros(1, dtype=int)
duration = 120 #120sec
hour_init = base_hour

b = np.zeros(1, dtype=int)
pAb = 1
delay_b = 30 #seconds
duration_b =120

c = np.zeros(1, dtype=int)
pAc = 0.6
delay_c = 30 #seconds
duration_c = 0


#Alarm Series Generation, A (cause), B(Effect). C(Effect)
while(occor_num < ocorr): 
    
    #A generation - begin
    srtd_hour = random.normalvariate(3, 1)
    srtd_hour_begin = hour_init + dt.timedelta(hours=srtd_hour)
    srtd_hour_end = srtd_hour_begin + dt.timedelta(seconds = duration)

    idx_init_a_occor = int(math.ceil((srtd_hour_begin - base_hour).total_seconds()/t_sample))
    idx_end_a_occor = int(math.ceil((srtd_hour_end - base_hour).total_seconds()/t_sample))
        
    if(idx_end_a_occor > a.size):
            a.resize(idx_end_a_occor)

    for i in np.arange(idx_init_a_occor, idx_end_a_occor + 1):
        a[i-1] = 1
    #A generation - end
         
    
    #B generation begin
    srtd_prob_b = random.uniform(0,1)  
    if srtd_prob_b <= pAb:
        srtd_hour_begin_b = srtd_hour_begin + dt.timedelta(seconds = delay_b)
        srtd_hour_end_b = srtd_hour_begin_b + dt.timedelta(seconds=duration_b)

        idx_init_b_occor = int(math.ceil((srtd_hour_begin_b - base_hour).total_seconds()/t_sample))
        idx_end_b_occor = int(math.ceil((srtd_hour_end_b - base_hour).total_seconds()/t_sample))

        if(idx_end_b_occor > b.size):
                b.resize(idx_end_b_occor)

        for j in np.arange(idx_init_b_occor, idx_end_b_occor +1):
            b[j-1] =  1  
    #B generation end

    #C generation begin
    srtd_prob_c = random.uniform(0,1)
    if srtd_prob_c <= pAc:
        srtd_hour_begin_c = srtd_hour_begin + dt.timedelta(seconds = delay_c)
        srtd_hour_end_c = srtd_hour_begin_c + dt.timedelta(seconds=duration_c)

        idx_init_c_occor = int(math.ceil((srtd_hour_begin_c - base_hour).total_seconds()/t_sample))
        idx_end_c_occor = int(math.ceil((srtd_hour_end_c - base_hour).total_seconds()/t_sample))
        
        if(idx_end_c_occor > c.size):
            c.resize(idx_end_c_occor)

        for j in np.arange(idx_init_c_occor, idx_end_c_occor +1):
            c[j-1] = 1    
    #C generation end
    
    hour_init = srtd_hour_begin
    occor_num = occor_num + 1

#Making series the same length
max_len = max(a, b, c, key=len).size
a = np.concatenate([a, np.zeros(max_len - a.size)])
b = np.concatenate([b, np.zeros(max_len - b.size)])
c = np.concatenate([c, np.zeros(max_len - c.size)])
sum(a)

30.0

In [326]:
tsample = t_sample
window_lag = 2 #shoud be minor than size of the series

a_first_diff = np.concatenate([[0],a])
b_first_diff = np.concatenate([[0],b])
c_first_diff = np.concatenate([[0],c])

corr_mat = np.zeros([3,3], dtype=float)
count_occor = 0;


#alrms = [a_first_diff, b_first_diff, c_first_diff]
alrms = [[1,0,1], [0,1,1], [1,0,0]]
for i, alm in enumerate(alrms):
    print(i)
    reduced_alarms = alrms.copy()
    reduced_alarms.pop(i)
    print(alrms)
    for j,el in enumerate(alm):
        print('currenty el', el)
        if el == 1:
            print('on if', alm, el)
            count_occor = count_occor + 1
            for k,r_alm in enumerate(reduced_alarms):
                print('k', k)
                print('alm intern', r_alm)
                counted = False
                for l in np.arange(j+1, j+window_lag):
                    if( l< len(r_alm)):
                        if r_alm[l] == 1 and  not counted:
                            counted = True
                            print('here')
                            corr_mat[i][alrms.index(r_alm)] = corr_mat[i][k]+1
    if count_occor > 0: 
        corr_mat[i] = corr_mat[i]/count_occor
        count_occor = 0
    
                        
                        
                        
                
                



0
[[1, 0, 1], [0, 1, 1], [1, 0, 0]]
currenty el 1
on if [1, 0, 1] 1
k 0
alm intern [0, 1, 1]
here
k 1
alm intern [1, 0, 0]
currenty el 0
currenty el 1
on if [1, 0, 1] 1
k 0
alm intern [0, 1, 1]
k 1
alm intern [1, 0, 0]
1
[[1, 0, 1], [0, 1, 1], [1, 0, 0]]
currenty el 0
currenty el 1
on if [0, 1, 1] 1
k 0
alm intern [1, 0, 1]
here
k 1
alm intern [1, 0, 0]
currenty el 1
on if [0, 1, 1] 1
k 0
alm intern [1, 0, 1]
k 1
alm intern [1, 0, 0]
2
[[1, 0, 1], [0, 1, 1], [1, 0, 0]]
currenty el 1
on if [1, 0, 0] 1
k 0
alm intern [1, 0, 1]
k 1
alm intern [0, 1, 1]
here
currenty el 0
currenty el 0


In [327]:
corr_mat

array([[0. , 0.5, 0. ],
       [0.5, 0. , 0. ],
       [0. , 1. , 0. ]])

In [323]:
a = [9,8,1]
a.pop(1)
a

[9, 1]